# Programação Paralela Híbrida: MPI + OpenMP Offloading

Autores: *Calebe P. Bianchini, Evaldo B. Costa, Gabriel P. Silva*

## Setup de ambiente

Para conseguir usar OpenMP em um ambiente híbrido (CPU + GPU), é necessário preparar o kit de ferramentas.

Antes de mais nada, verifique se seu ambiente de Google Colab está com o tipo de Runtime usando alguma GPU - veja nos menus disponíveis neste Lab.

1. Instale um compilador que consiga usar OpenMP + GPU, como, por exemplo, o GCC-13, e fez o ajuste do ambiente para usá-lo.

In [ ]:
!sudo add-apt-repository -y ppa:ubuntu-toolchain-r/test

PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/ubuntu-toolchain-r/test/ubuntu/ jammy main'
Description:
Toolchain test builds; see https://wiki.ubuntu.com/ToolChain

More info: https://launchpad.net/~ubuntu-toolchain-r/+archive/ubuntu/test
Adding repository.
Found existing deb entry in /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding deb entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Found existing deb-src entry in /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding key to /etc/apt/trusted.gpg.d/ubuntu-toolchain-r-ubuntu-test.gpg with fingerprint C8EC952E2A0E1FBDC5090F6A2C277A0A352154E5
Get:1 file:/var/cuda-repo-wsl-ubuntu-12-6-local  InRelease [1572 B]
Get:1 file:/var/cuda-repo-wsl-ubuntu-12-6-local  InRelease [1572 B]
Hit:2 https://developer.do

In [ ]:
!sudo apt install -y gcc-13 g++-13 gcc-13-offload-nvptx libgomp1

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++-13 is already the newest version (13.1.0-8ubuntu1~22.04).
gcc-13 is already the newest version (13.1.0-8ubuntu1~22.04).
gcc-13-offload-nvptx is already the newest version (13.1.0-8ubuntu1~22.04).
libgomp1 is already the newest version (13.1.0-8ubuntu1~22.04).
libgomp1 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [ ]:
!sudo ln -sfnv /usr/bin/gcc-13 /usr/bin/gcc

'/usr/bin/gcc' -> '/usr/bin/gcc-13'


2. Verifique se os compiladors para OpenMP e GPU estão todos disponíveis. No nosso caso, usaremos o GCC-13 e a versão já instalada do NVidia CUDA:

In [ ]:
!gcc --version
!nvcc --version
!nvidia-smi

gcc (Ubuntu 13.1.0-8ubuntu1~22.04) 13.1.0
Copyright (C) 2023 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Fri_Jun_14_16:34:21_PDT_2024
Cuda compilation tools, release 12.6, V12.6.20
Build cuda_12.6.r12.6/compiler.34431801_0
gcc: error: unrecognized command-line option ‘--showme:version’
Wed Jan 29 16:04:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.134                Driver Version: 553.35         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compu

3. Teste seu ambiente

In [ ]:
%%writefile test.c

#include <stdio.h>
#include <omp.h>

int main() {
  int numdevices = omp_get_num_devices();
  int device= omp_get_device_num();
  int max_threads = omp_get_max_threads();
  printf("number of devices= %d *** device= %d\n", numdevices, device);
  printf("max number of threads: %d\n", max_threads);
}

Writing test.c


O resultado deverá ser o número de núcleos disponíveis no seu computador e a quantidade de GPUS.

In [ ]:
!gcc -fopenmp test.c -o test
!./test

number of devices= 1 *** device= 1
max number of threads: 22


Verificando o tempo de execução da versão sequencial.

In [ ]:
!gcc -o ./bin/calc_pi ./src/calc_pi.c
!./bin/calc_pi

pi = 3.141592653589451, 10000000000 passos, computados em 18.467919 segundos


Agora verificando o código da versão com OpenMP offloading, que faz a descarga do laço para execução no acelerador (GPU).

In [ ]:
! cat src/omp_off_calc_pi.c


#include <stdio.h>
#include <omp.h>
#include <math.h>
int main() {
  long int num_passos = 10000000000; // Número de passos para a integração
  double passo = 1.0 / (double) num_passos;
  double pi = 0.0, inicio, fim;
  inicio = omp_get_wtime();
  #pragma omp target data map(tofrom: pi) map(to:num_passos, passo) device(1)// Diretiva para offloading para a GPU
  #pragma omp target teams distribute parallel for reduction(+:pi) // Paralelização com OpenMP
     for (long int i = 0; i < num_passos; i++) {
          double x = (i + 0.5) * passo;
          pi += 4.0 / (double) (1.0 + x * x); 
     }
  pi *= passo;
  fim = omp_get_wtime();
  printf("Valor de Pi calculado: %2.15f\n", pi);
  printf("Tempo de execução: %f segundos\n", fim - inicio);
  return 0;
}


Compilando e executando com as opções necessárias. Veja a diferença no tempo de execução.

In [ ]:
!gcc -fopenmp -fno-lto -fstack-protector -o bin/omp_off_calc_pi src/omp_off_calc_pi.c
!./bin/omp_off_calc_pi

Valor de Pi calculado: 3.141592653589809
Tempo de execução: 2.376862 segundos


Verificando o código com MPI e OpenMP threads para execução do cálculo de Pi.

In [ ]:
!cat src/mpi_omp_calc_pi.c

#include <stdio.h>
#include "mpi.h"
#include <omp.h>
static long num_passos = 10000000000;
double passo;
int main(int argc, char *argv[])
{
  int ranque, numprocs, provided;
  double x, pi, soma = 0.0, soma_global = 0.0;
  double inicio, tempo;
  // Inicia o MPI com suporte para threads
  MPI_Init_thread(&argc, &argv, MPI_THREAD_FUNNELED, &provided);
  if (provided < MPI_THREAD_FUNNELED) {
    printf("Nível de suporte para threads não é suficiente!\n");
    MPI_Abort(MPI_COMM_WORLD, 1);
  }
  MPI_Comm_rank(MPI_COMM_WORLD, &ranque); // O rank do processo
  MPI_Comm_size(MPI_COMM_WORLD, &numprocs); // O número de processos
  passo = 1.0 / (double)num_passos;
  inicio = omp_get_wtime(); // Tempo de início da execução
// OpenMP com paralelismo de threads dentro do processo
#pragma omp parallel for private(x) shared(ranque, num_passos, numprocs, passo) reduction(+:soma) num_threads(4)
  for (long int i = ranque; i < num_passos; i += numprocs) { // Saltos de acordo com o número de processos


Compilando e executando com MPI e OpenMP threads.

In [ ]:
!mpicc -fopenmp -fno-lto -fstack-protector src/mpi_omp_calc_pi.c -Wall -o bin/mpi_omp_calc_pi
!mpirun -np 4 bin/mpi_omp_calc_pi

Processo 2 Redução 7853981633.724379 
Processo 0 Redução 7853981634.724530 
Processo 3 Redução 7853981633.224398 
Processo 1 Redução 7853981634.224747 
pi = 3.141592653589806, 10000000000 passos, computados em 2.177759 segundos


In [ ]:
!cat src/mpi_omp_off_calc_pi.c

#include <stdio.h>
#include "mpi.h"
#include <omp.h>
static long num_passos = 10000000000;
double passo;
int main(int argc, char *argv[])
{
  int ranque, numprocs, provided;
  double x, pi, soma = 0.0, soma_global = 0.0;
  double inicio, tempo;
  // Inicia o MPI com suporte para threads
  MPI_Init_thread(&argc, &argv, MPI_THREAD_FUNNELED, &provided);
  if (provided < MPI_THREAD_FUNNELED) {
    printf("Nível de suporte para threads não é suficiente!\n");
    MPI_Abort(MPI_COMM_WORLD, 1);
  }
  MPI_Comm_rank(MPI_COMM_WORLD, &ranque); // O rank do processo
  MPI_Comm_size(MPI_COMM_WORLD, &numprocs); // O número de processos
  passo = 1.0 / (double)num_passos;
  inicio = omp_get_wtime(); // Tempo de início da execução
// Offloading com OpenMP para o acelerador (GPU), com paralelismo dentro do processo
#pragma omp target data map(tofrom:soma) map(to:numprocs, num_passos, ranque, passo) map(alloc:x) device(1)
#pragma omp target teams distribute parallel for reduction(+:soma) 
  for (long int

Compilando e executando com MPI + OpenMP Offloading.

In [ ]:
!mpicc -fopenmp -fno-lto -fstack-protector -o bin/mpi_omp_off_calc_pi src/mpi_omp_off_calc_pi.c
!mpirun -np 4 ./bin/mpi_omp_off_calc_pi

pi = 3.141592653589794, 10000000000 passos, computados em 2.240136 segundos


In [ ]:
!cat src/mpi_omp_bal_calc_pi.c

#include <stdio.h>
#include "mpi.h"
#include <omp.h>
#include <stdlib.h>
#include <time.h>
// Defina a proporção da carga de trabalho para a GPU (em %)
#define GPU_WORKLOAD 70
static long num_passos = 10000000000; // Número total de pontos aleatórios
int main(int argc, char *argv[]) {
    long int gpu_contagem_local = 0, cpu_contagem_local = 0, contagem_global = 0;
    int ranque, numprocs, provided;
    double x, y, z, pi;
    double inicio, tempo;
    // Inicializa o MPI com suporte para threads
    MPI_Init_thread(&argc, &argv, MPI_THREAD_FUNNELED, &provided);
    if (provided < MPI_THREAD_FUNNELED) {
        printf("Nível de suporte para threads não é suficiente!\n");
        MPI_Abort(MPI_COMM_WORLD, 1);
    }
    MPI_Comm_rank(MPI_COMM_WORLD, &ranque); // O rank do processo
    MPI_Comm_size(MPI_COMM_WORLD, &numprocs); // O número de processos
    // Inicializa o gerador de números aleatórios
    srand(time(NULL) + ranque);
    // Define a carga de trabalho para a GPU e para as t

In [ ]:
!mpicc -fopenmp -fno-lto -fstack-protector -o bin/mpi_omp_bal_calc_pi src/mpi_omp_bal_calc_pi.c
!mpirun -np 4 ./bin/mpi_omp_bal_calc_pi

pi = 3.141827072000000, 1000000000 amostras, computados em 44.408770 segundos


#Números Primos
O programa em questão serve para determinar a quantidade de números primos entre 0 e um determinado valor inteiro N.
Embora possa parecer um programa trivial a princípio, ele tem algumas particularidades que o tornam um problema interessante.
Na matemática, o Teorema do Número Primo (TNP) descreve a distribuição assintótica dos números primos entre os inteiros positivos.
Ele formaliza a ideia intuitiva de que os números primos tornam-se menos comuns à medida que N aumenta, quantificando precisamente a taxa em que isso ocorre.
<p>
Bom, a nossa primeira tentativa de paralelização seria dividir o total de N números igualmente entre os P processadores disponíveis, ou seja, N/P valores para cada uma dos processos ou threads.
No entanto, há uma implicação importante: a distribuição dos números primos não é uniforme entre os inteiros.
Esse fato mostra que a divisão direta é ineficaz, pois os processos ou threads que receberem intervalos com uma maior concentração de números primos terão uma carga de trabalho significativamente maior.

In [4]:
!cat src/seq_primos.c

#include <stdio.h>
#include <stdbool.h>
#include <math.h>

// Função para verificar se um número é primo
bool is_prime(int num) {
    if (num <= 1) return false;
    if (num == 2) return true;
    if (num % 2 == 0) return false;

    for (int i = 3; i <= sqrt(num); i += 2) {
        if (num % i == 0) return false;
    }
    return true;
}

int main() {
    long int N, total_primes=0;
    printf("Digite o valor de N: ");
    scanf("%ld", &N);

    // Dividir o trabalho entre CPU e GPU
    for (int i = 1; i <= N ; i+=2) {
         if (is_prime(i)) {
             total_primes++;
         }
    }
    total_primes++;  // O número 2 também é primo

    // Soma os resultados da CPU e GPU

    printf("Total de números primos entre 1 e %ld: %ld\n", N, total_primes);
    return 0;
}


In [14]:
!mpicc -fopenmp -o bin/seq_primos src/seq_primos.c
!time ./bin/seq_primos 100000000

Total de números primos entre 1 e 100000000: 5761455

real	1m5.311s
user	1m5.312s
sys	0m0.000s


#Versão com OpenMP offloading

A seguir a versão com OpenMP offloading, com balanceamento de carga para os núcleos também.

In [7]:
!cat src/omp_off_primos.c

#include <stdio.h>
#include <stdbool.h>
#include <stdlib.h>
#include <math.h>
#include <omp.h>

// Função para verificar se um número é primo
bool is_prime2(int num) {
    for (int i = 3; i <= sqrt(num); i += 2) {
        if (num % i == 0) return false;
    }
    return true;
}

bool is_prime(long int n) {
    if (n <= 3) return true;
    if (n % 2 == 0 || n % 3 == 0) return false;
    for (long int i = 5; i * i <= n; i += 6) {
        if (n % i == 0 || n % (i + 2) == 0) return false;
    }
    return true;
}

int main(int argc, char *argv[]) {
int N;
    if (argc < 2) {
        printf("Valor inválido! Entre com o valor do maior inteiro\n");
       	return 0;
    } else {
        N = strtol(argv[1], (char **) NULL, 10);
    }

    int total_primes_cpu = 0;
    int total_primes_gpu = 0;

    // Dividir o trabalho entre CPU e GPU
    int split_point = (int)(N * .3); // 30% para CPU, 70% para GPU
    if (split_point % 2 != 0) 
        split_point--; // Se for ímpar, subtrai 1 para tornar 

In [9]:
!gcc -fopenmp -fno-lto -fstack-protector -o bin/omp_off_primos src/omp_off_primos.c -lm
!./bin/omp_off_primos 100000000

Total de números primos entre 1 e 100000000: 5761455. Calculado em 13.524612 segundos.
Primos encontrados na CPU: 1857858
Primos encontrados na GPU: 3903596
Ponto de Divisão: 30000000


#Versão com MPI e  OpenMP offloading

A seguir a versão com MPI e OpenMP offloading, com balanceamento de carga para os núcleos também.

In [10]:
!cat src/mpi_off_primos.c

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <omp.h>
#include <mpi.h>

int is_prime(long int n) {
    if (n <= 1) return 0;
    if (n <= 3) return 1;
    if (n % 2 == 0 || n % 3 == 0) return 0;
    for (long int i = 5; i * i <= n; i += 6) {
        if (n % i == 0 || n % (i + 2) == 0) return 0;
    }
    return 1;
}
#define GPU_WORKLOAD 70

int main(int argc, char *argv[]) {
    int numprocs, rank, salto;
    long int num_primes_gpu = 0, num_primes_cpu = 0, global_num_primes = 0;
    long int  n, i, gpu_end;
    double start_time, end_time;

    MPI_Init(&argc, &argv);
    MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    if (argc < 2) {
        printf("Valor inválido! Entre com o valor do maior inteiro\n");
       	return 0;
    } else {
        n = strtol(argv[1], (char **) NULL, 10);
    }

    gpu_end = n  * GPU_WORKLOAD / 100;
    if (gpu_end % 2 != 0) {
        gpu_end--; // Se for ímpar, subtrai 1 para tornar par
    }
 

In [12]:
!mpicc -fopenmp -fno-lto -fstack-protector -o bin/mpi_off_primos src/mpi_off_primos.c
!mpirun -np 4 ./bin/mpi_off_primos 100000000

Total local 2  1440529
Total local 1  1440666
Total local 0  1440602
Total local 3  1439657
Total de primos até 100000000: 5761455
Calculado em 8.697654 segundos
